<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import re
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding,GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk import word_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
data_url = 'https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv'
data = pd.read_csv(data_url, sep='\t')

In [0]:
data.head(5)

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


In [0]:
y = data["PRODUCT_ID"]
X = data["cleaned_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Задание 1

In [0]:
clf1 = LogisticRegression(multi_class='multinomial', solver='sag')
clf2 = RandomForestClassifier(n_estimators=150)
clf3 = GaussianNB()
clf4 = LinearSVC(max_iter = 500)

eclf = VotingClassifier(estimators=[
        ('lr', clf1), 
        ('rf', clf2), ('gnb', clf3), ('svc', clf4)], voting='hard')

voting = Pipeline([
                   ('feats', FeatureUnion([
                                           ('count', CountVectorizer(min_df = 7, max_df = 0.7)),
                                           ('tfidf', TfidfVectorizer(min_df = 30, max_df = 0.2))                                           
                                           ])),
                   ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
    ('clf', eclf)
    ])
voting = voting.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
predictions = voting.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.71
Recall:   0.70
F1-measure:   0.70
Accuracy:   0.70


## Задание 2

In [0]:
clf1 = LogisticRegression(multi_class='multinomial', solver='sag')
clf2 = RandomForestClassifier(n_estimators=150)
clf3 = GaussianNB()
clf4 = LinearSVC(max_iter = 500)
clf5 = DecisionTreeClassifier()
bagging1 = BaggingClassifier(base_estimator=clf5, n_estimators=20, max_samples=0.8, max_features=0.8)

clf6 = DecisionTreeClassifier(criterion='entropy', max_depth=2)
boosting = AdaBoostClassifier(base_estimator=clf6, n_estimators=20)

boosting2 = xgb.XGBClassifier(objective='multi:softmax', n_jobs=-1, max_depth=6)

all_voting = VotingClassifier(estimators=[
        ('clf1', clf1), 
        ('clf2', clf2), 
        ('clf3', clf3), 
        ('clf4', clf4), 
        ('bag', bagging1), 
        ('boo1', boosting), 
        ('boo2', boosting2)
        ], 
        voting='hard')

In [0]:
pipe = Pipeline([
                 ('feats', FeatureUnion([
                                         ('count', CountVectorizer(min_df = 7, max_df = 0.7)),
                                         ('tfidf', TfidfVectorizer(min_df = 30, max_df = 0.2))
                                         ])),
                 ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                 ('clf', all_voting)
                 ])
pipe = pipe.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
predictions = pipe.predict(X_test)
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

Precision:   0.74
Recall:   0.72
F1-measure:   0.73
Accuracy:   0.72
